<a href="https://colab.research.google.com/github/MarijaGijic/Julia_kernel_abstractions/blob/main/2D_dilation_erosion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [6]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


In [7]:
using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   2695.6 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 12 seconds. 460 already precompiled.


  256.226 ms (2 allocations: 32.00 MiB)


In [8]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

Tue Apr  8 19:14:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [10]:
using Pkg
Pkg.add("CUDA")
Pkg.add("AMDGPU")
Pkg.add("KernelAbstractions")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
   Installed InitialValues ────── v0.3.1
   Installed BangBang ─────────── v0.4.4
   Installed CompositionsBase ─── v0.1.2
   Installed ROCmDeviceLibs_jll ─ v5.6.1+1
   Installed LLVM_jll ─────────── v15.0.7+10
   Installed StableTasks ──────── v0.1.7
   Installed Accessors ────────── v0.1.42
   Installed OhMyThreads ──────── v0.8.2
   Installed ChunkSplitters ───── v3.1.2
   Installed TaskLocalValues ──── v0.1.2
   Installed AMDGPU ───────────── v1.2.7
   Installed AcceleratedKernels ─ v0.3.3
    Updating `~/.julia/environments/v1.10/Project.toml`
  [21141c5a] + AMDGPU v1.2.7
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [21141c5a] + AMDGPU v1.2.7
  [6a4ca0a5] + AcceleratedKernels v0.3.3
  [7d9f7c33] + Accessors v0.1.42
  [198e06fe] + BangBang v0.4.4
  [ae650224] + ChunkSplitters v3.1.2
  [

In [22]:
abstract type AbstractImageNew end;

struct Image2D_{T} <: AbstractImageNew
    height_::T
    width_::T

end

struct Image3D_{T} <: AbstractImageNew
    height_::T
    width_::T
    depth_::T

end

function create_rand_image_2D(image::Image2D_{Int64})
    h = image.height_
    w = image.width_
    return rand(Bool, h, w)

end

function create_rand_image_3D(image::Image3D_{Int64})
    h = image.height_
    w = image.width_
    d = image.depth_
    return rand(Bool, h, w, d)
end

create_rand_image_3D (generic function with 1 method)

In [23]:
using KernelAbstractions, Test

#------------------------------------dilation---------------------------------------

@kernel function dilate_kernel!(output_img, input_img, struct_element)
    I = @index(Global, Cartesian)
    i, j = Tuple(I)
    offset_h = div(size(struct_element)[1], 2)
    offset_w = div(size(struct_element)[2], 2)

    if i <= size(input_img, 1) && j <= size(input_img, 2)
        result = false

        for m in 1:size(struct_element)[1]
            for n in 1:size(struct_element)[2]
                ni = i + m - offset_h - 1
                nj = j + n - offset_w - 1

                # Only check input if it's in bounds
                if 1 <= ni <= size(input_img)[1] && 1 <= nj <= size(input_img)[2]
                    if input_img[ni, nj] == 1 && struct_element[m, n] == 1
                        result = true
                        break
                    end
                end

            end
            if result
                break
            end
        end

        @inbounds output_img[i, j] = result ? 1 : 0
    end
end


function dilate_2D!(output, img_input, struct_element)

    backend = get_backend(img_input)
    kernel! = dilate_kernel!(backend)
    kernel!(output, img_input, struct_element, ndrange = size(output))

    return output
end



dilate_2D! (generic function with 1 method)

In [30]:
#------------------------------------erosion---------------------------------------

@kernel function erode_kernel!(output_img, input_img, struct_element)
    I = @index(Global, Cartesian)
    i, j = Tuple(I)
    offset_h = div(size(struct_element)[1], 2)
    offset_w = div(size(struct_element)[2], 2)

    if i <= size(input_img, 1) && j <= size(input_img, 2)
        result = true

        for m in 1:size(struct_element)[1]
            for n in 1:size(struct_element)[2]
                ni = i + m - offset_h - 1
                nj = j + n - offset_w - 1

                if 1 <= ni <= size(input_img)[1] && 1 <= nj <= size(input_img)[2]
                    if struct_element[m, n] == 1 && input_img[ni, nj] == 0
                        result = false
                        break
                    end
                else
                    # Out-of-bounds counts as 0 in erosion
                    if struct_element[m, n] == 1
                        result = false
                        break
                    end
                end
            end
            if !result
                break
            end
        end

        @inbounds output_img[i, j] = result ? 1 : 0
    end
end


function erode_2D!(output, img_input, struct_element)

    backend = get_backend(img_input)
    kernel! = erode_kernel!(backend)
    kernel!(output, img_input, struct_element, ndrange = size(output))

    return output
end


erode_2D! (generic function with 1 method)

In [35]:
function testing()
    # === Input for dilation (CPU & GPU)
    img_input_dilation = Bool[
        0 0 0 0 0 0 0 0 0 0 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 0 0 0 0 0 0 0 0 0 0;
    ]

    expected_result_dilation = Bool[
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 0 0;
        1 1 1 1 1 1 1 1 1 0 0;
    ]

    # === Input for erosion (CPU & GPU)
    img_input_erosion = Bool[
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
    ]

    expected_result_erosion = Bool[
        0 0 0 0 0 0 0 0 0 0 0 0 0;
        0 1 1 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 1 1 0;
        0 0 0 0 0 0 0 0 0 0 0 0 0;
    ]

    # Structuring element
    struct_element = ones(Bool, 3, 3)

    # Output arrays (CPU)
    output_dilation = zeros(Bool, size(expected_result_dilation))
    output_erosion = zeros(Bool, size(expected_result_erosion))

    # Call CPU versions
    dilate_2D!(output_dilation, img_input_dilation, struct_element)
    erode_2D!(output_erosion, img_input_erosion, struct_element)

    @test output_dilation == expected_result_dilation
    @test output_erosion == expected_result_erosion

    # GPU versions
    d_img_input_dilation = CuArray(img_input_dilation)
    d_img_input_erosion = CuArray(img_input_erosion)
    d_struct_element = CuArray(struct_element)

    d_output_dilation = CuArray(zeros(Bool, size(expected_result_dilation)))
    d_output_erosion = CuArray(zeros(Bool, size(expected_result_erosion)))

    dilate_2D!(d_output_dilation, d_img_input_dilation, d_struct_element)
    erode_2D!(d_output_erosion, d_img_input_erosion, d_struct_element)

    # Bring results back from GPU
    gpu_result_dilation = Array(d_output_dilation)
    gpu_result_erosion = Array(d_output_erosion)

    @test gpu_result_dilation == expected_result_dilation
    @test gpu_result_erosion == expected_result_erosion
end

testing (generic function with 1 method)

In [36]:
testing()

Test Passed